# Generacion de los metodos de downscaling estadistico 

A contunuación se expondra todo el proceso llevado acabo para la elaboración del downscaling estadístico para la region de América del Sur. Usaremos el Subset numero 2 que es el que hemo decidido usar por que es el mejor en cuanto a su reproductividad local.

### Importamos las librerias que utilizaremos 

In [1]:
library(loadeR)
library(magrittr)
require(transformeR)
require(downscaleR)
require(visualizeR)
require(climate4R.value)
require(downscaleR.keras)
library(ggplot2)

Loading required package: rJava

Loading required package: loadeR.java

Java version 11x amd64 by Oracle Corporation detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.3 (2021-07-05) is loaded


Get the latest stable version (0.2.4) using <devtools::install_github('SantanderMetGroup/climate4R.UDG')>

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.7.0 (2020-09-18) is loaded


Get the latest stable version (1.7.1) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.

Loading required package: transformeR




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.1.3 (2021-08-04) is loaded


Get the latest stable version (2.1.4) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: downscaleR

downscaleR version 3.3.3 (2021-07-05) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: visualizeR

Warning message:
“no DISPLAY variable so Tk is not available”
visualizeR version 1.6.1 (2021-03-11) is loaded

Please see 'citation("visualizeR")' to cite this package.

Loading required package: climate4R.value

Loading required package: VALUE

---------------------------------------------- 
| VALUE version 2.2.2 (2021-08-18) is loaded |
|         http://www.value-cost.eu           |
----------------------------------------------

Loading required package: downscaleR.keras

Loading required package: keras

Loading required package: tensorflow



- Definimos las variables independientes. 

In [ ]:
var.list <- c('ta@500',
              'ta@700',
              'ta@850',
              'z@700',
              'z@850',
              'z@500',
              'ua@850',
              'va@850')

- Importamos dichas variables y generamos un archivo con ellas.

In [ ]:
grid.list <- lapply(var.list, function(x){
    loadGridData(dataset = "path_to_VALUE_ERA-INTERIM_data",#http://www.value-cost.eu/data
                 var = x,
                 # lonLim = c(8,32),
                 # latLim = c(38,42),
                 years = 1979:2008)
    }
)
x <- makeMultiGrid(grid.list)

In [ ]:
y <- loadStationData(dataset = "path_to_VALUE_station_data",#http://www.value-cost.eu/data
                 var = "tasmax",
                 # lonLim = c(8,32),
                 # latLim = c(38,42),
                 years = 1979:2008)
    


### Modelos lineales generalizados

Comenzaremos con 3 GLMs, dependiendo de los vecinos cercanos que se seleccionen. En este caso se han selecionado 3 escenarios distintos, 1 vecino cercano (GLM_1), 4 vecinos cercanos (GLM_4), 16 vecinos cercanos (GLM_16). 

- Definimos los folds 

In [ ]:
folds <- list(1979:1989, 1990:1999, 2000:2008)

- Estandarizamos las variables

In [ ]:
scaling.pars <- list(type = "standardize",
                    spatial.frame = "gridbox")

In [ ]:
scaleGrid.args = scaling.pars

#### GLM1

- La funcion local.pars.M1 te permite selecionar los vecinos cercanos, en nuetro casi 1

In [ ]:
local.pars.M1 <- list(n=1, vars=var.list)

- Generamos la cross-validacion del modelo 

In [ ]:
GLM1cv <- downscaleCV(x = x, y = y, method="GLM",
                   family=gaussian(link = "identity"),
                   folds=folds,
                   scaleGrid.args = scaling.pars,
                   prepareData.args = list(global.vars = NULL,
                                          local.predictors= local.pars.M1,
                                          spatial.predictors= NULL))

#### GLM4

- Seguimos los mismos pasos que en GLM1 pero ahora con 4 vecinos cercanos 

In [ ]:
local.pars.M4 <- list(n=4, vars=var.list)

In [ ]:
GLM4cv <- downscaleCV(x = x, y = y, method="GLM",
                   family=gaussian(link = "identity"),
                   folds=folds,
                   scaleGrid.args = scaling.pars,
                   prepareData.args = list(global.vars = NULL,
                                          local.predictors= local.pars.M4,
                                          sparial.predictors= NULL))

#### GLM16

- Generamos el GLM para 16 vecinos cercanos

In [ ]:
local.pars.M16 <- list(n=16, vars=var.list)

In [ ]:
GLM16cv <- downscaleCV(x = x, y = y, method="GLM",
                   family=gaussian(link = "identity"),
                   folds=folds,
                   scaleGrid.args = scaling.pars,
                   prepareData.args = list(global.vars = NULL,
                                          local.predictors= local.pars.M16,
                                          sparial.predictors= NULL))

- Para comprobar que los modelos estan bien generados realizaremos una representacion grafica para ver realmente es asi

- Definimos la funcion que agregara los parametros mediante la media.

In [ ]:
aggr.pars <- list(FUN = "mean", na.rm=TRUE)

- Agregamos los datos del GLM1cv mensualmente 

In [ ]:
pred.GLM1 <- aggregateGrid(GLM1cv, aggr.m= aggr.pars)

- Agregamos los datos de las observacion mensualmente

In [ ]:
obs <- aggregateGrid(y, aggr.m= aggr.pars)

- Con la funcion temporalPlot representamos la observacion y la prediccion. 

In [ ]:
temporalPlot(pred.M1, obs,lwd=0.9,
            xyplot.custom = list(xlab = "año",
                                ylab = "temp",
                                scale = list(cex =1,
                                            x = list(rot=0))))

Observamos que si hay una correlacion entre las observaciones y las predicciones, por lo que el modelo esta bien generado.

#### Redes Neuronales

Una vez tenemos generados todos los GLMs realizaremos el mismo proceso pero en este caso seran 3 redes neuronales, las cuales tambien obtendran informacion de 1,4,16 vecinos cercanos respectivamente. Por ultimo, acabaremos con una red neuronal convolucinal.

#### NN1 

In [ ]:
pred <- lapply(1:getShape(y,"loc"),FUN = function(z) {
  y <- subsetGrid(y, station.id = y$Metadata$station_id[z]) %>% redim(member = FALSE, loc = TRUE)
  l0 <- layer_input(shape = 8) 
  l1 <- layer_dense(l0, units = 50, activation = "relu")
  l2 <- layer_dense(l1, units = 50, activation = "relu")  
  l3 <- layer_dense(l2, units = 1, activation = 'linear')
  
  model <- keras_model(inputs = l0, outputs = l3)
  pred <- downscaleCV.keras(x,y,model,
                            folds = folds,
                            prepareData.keras.args = list(first.connection = "dense", last.connection = "dense", channels = "last",local.predictors= local.pars.M1),
                            compile.args = list(loss = "mse", optimizer = optimizer_adam(lr = 0.001)),
                            scaleGrid.args = list("type" = "standardize"),
                            loss = "mse",
                            fit.args = list(batch_size = 100, epochs = 10000, validation_split = 0.1,verbose = 1,
                                            callbacks = list(callback_early_stopping(patience = 30))))
  k_clear_session(); rm(model)
  return(pred)
}) %>% bindGrid(dimension = "loc")    
nnss1 <- subsetGrid(pred, var = "tmax")
save(nnss1, file = "nn-ss1.rda")

#### NN4

In [ ]:
pred <- lapply(1:getShape(y,"loc"),FUN = function(z) {
  y <- subsetGrid(y, station.id = y$Metadata$station_id[z]) %>% redim(member = FALSE, loc = TRUE)
  l0 <- layer_input(shape = 32) 
  l1 <- layer_dense(l0, units = 50, activation = "relu")
  l2 <- layer_dense(l1, units = 50, activation = "relu")  
  l3 <- layer_dense(l2, units = 1, activation = 'linear')
  
  model <- keras_model(inputs = l0, outputs = l3)
  pred <- downscaleCV.keras(x,y,model,
                            folds = folds,
                            prepareData.keras.args = list(first.connection = "dense", last.connection = "dense", channels = "last",local.predictors= local.pars.M4),
                            compile.args = list(loss = "mse", optimizer = optimizer_adam(lr = 0.001)),
                            scaleGrid.args = list("type" = "standardize"),
                            loss = "mse",
                            fit.args = list(batch_size = 100, epochs = 10000, validation_split = 0.1,verbose = 1,
                                            callbacks = list(callback_early_stopping(patience = 30))))
  k_clear_session(); rm(model)
  return(pred)
}) %>% bindGrid(dimension = "loc")    
nnss4 <- subsetGrid(pred, var = "tmax")
save(nnss4, file = "nn-ss4.rda")

#### NN16

In [ ]:
pred <- lapply(1:getShape(y,"loc"),FUN = function(z) {
  y <- subsetGrid(y, station.id = y$Metadata$station_id[z]) %>% redim(member = FALSE, loc = TRUE)
  l0 <- layer_input(shape = 96) 
  l1 <- layer_dense(l0, units = 50, activation = "relu")
  l2 <- layer_dense(l1, units = 50, activation = "relu")  
  l3 <- layer_dense(l2, units = 1, activation = 'linear')
  
  model <- keras_model(inputs = l0, outputs = l3)
  pred <- downscaleCV.keras(x,y,model,
                            folds = folds,
                            prepareData.keras.args = list(first.connection = "dense", last.connection = "dense", channels = "last",local.predictors= local.pars.M16),
                            compile.args = list(loss = "mse", optimizer = optimizer_adam(lr = 0.001)),
                            scaleGrid.args = list("type" = "standardize"),
                            loss = "mse",
                            fit.args = list(batch_size = 100, epochs = 10000, validation_split = 0.1,verbose = 1,
                                            callbacks = list(callback_early_stopping(patience = 30))))
  k_clear_session(); rm(model)
  return(pred)
}) %>% bindGrid(dimension = "loc")    
nnss16 <- subsetGrid(pred, var = "tmax")
save(nnss16, file = "nn-ss16.rda")

#### CNN

- A continuacion creamos la red convolucional.

In [ ]:
pred <- lapply(1:getShape(y,"loc"),FUN = function(z) {
  y <- subsetGrid(y, station.id = y$Metadata$station_id[z]) %>% redim(member = FALSE, loc = TRUE)
  l0 <- layer_input(shape = c(getShape(x,"lat"),getShape(x,"lon"),getShape(x,"var")))
  l1 <- layer_conv_2d(l0 ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "same")
  l2 <- layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "same")
  l3 <- layer_conv_2d(l2,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "same")
  l4 <- layer_flatten(l3)    
  l5 <- layer_dense(l4, units = 1, activation = 'linear')
  
  model <- keras_model(inputs = l0, outputs = l5)
  pred <- downscaleCV.keras(x,y,model,
                            folds = folds,
                            prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                            compile.args = list(loss = "mse", optimizer = optimizer_adam(lr = 0.001)),
                            scaleGrid.args = list("type" = "standardize"),
                            loss = "mse",
                            fit.args = list(batch_size = 100, epochs = 10000, validation_split = 0.1,verbose = 1,
                                            callbacks = list(callback_early_stopping(patience = 30))))

  k_clear_session(); rm(model)
  return(pred)
}) %>% bindGrid(dimension = "loc")    
cnnss <- subsetGrid(pred, var = "tmax")
save(cnnss, file = "cnn-ss.rda")

### Analogos

- Creamo el modelo de analogos.

In [ ]:
Analogoscv <- downscaleCV(x = x, y = y, method="analogs",
                   folds=folds,
                   scaleGrid.args = scaling.pars,
                   prepareData.args = list(global.vars = NULL,
                                          
                                          sparial.predictors= NULL))